# FSRQ SED model fit (NO BH mass estimation)

In [2]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import pandas as pd
import os
import astropy.units as u
from astropy.constants import c, G, k_B, m_e, M_sun
from astropy.coordinates import Distance
from pathlib import Path
from astropy.table import Table
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
%matplotlib widget

# import agnpy classes
import agnpy
from agnpy.emission_regions import Blob
from agnpy.spectra import BrokenPowerLaw
from agnpy.synchrotron import Synchrotron
from agnpy.compton import SynchrotronSelfCompton, ExternalCompton
from agnpy.targets import RingDustTorus, SSDisk
from agnpy.utils.plot import load_mpl_rc, sed_x_label, sed_y_label, plot_sed

load_mpl_rc()

# import sherpa classes
from sherpa.models import model
from sherpa import data
from sherpa.fit import Fit
from sherpa.stats import Chi2
from sherpa.optmethods import LevMar

from AgnpyEC import AgnpyEC


In [3]:
###
def ClearData():
    data = {
        'i': [],
        'm': [],
        'j': [],
        'k': [],
        'k_e': [],
        'p1': [],
        'p2': [],
        'gamma_b': [],
        'gamma_min': [],
        'gamma_max': [],
        'B': [],
        'delta_D': [],
        'R_b': [],
        'chi_2': [],
        'V_b': [],
        't_var':[],
        'd_L': [],
        'Gamma': [],
        'Beta': [],
        'xi': [],
        'theta_s': [],
        'xi_dt': [],
        'T_dt': [],
        'R_dt': [],
    }
    return data

def SED_Data():
    SED_data = {
        'fit_x': [],
        'fit_y': [],
        'syn_y': [],
        'ssc_y': [],
        'ec_dt_y': [],
        'disk_bb_y': [],
        'dt_bb_y': [],
    }
    return SED_data
    
### read in all sources
source_table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED progress/spectral index New 2022_10_18.csv') 
source_name = source_table.Name.values
z = source_table.z.values
source_class = source_table.class_name.values
source_alpha = source_table.alphaRadK.values

QSOind = []
BL_Lac_can_ind = []
BL_Lac_galaxy_dom_ind = []
BL_Lac_ind = []
Blazar_ind = []
classclassify = [Blazar_ind, BL_Lac_can_ind, BL_Lac_galaxy_dom_ind, QSOind, BL_Lac_ind]
classnameall  = [' Blazar Uncertain type', ' BL Lac Candidate', ' BL Lac-galaxy dominated', ' QSO RLoud flat radio sp.', ' BL Lac']
    
for j in range(0, 5): # 1367
    classclassify[j] = np.where( source_class == classnameall[j] )[0]
    
source_name_new = source_name[classclassify[3]]
z_new = z[classclassify[3]]
alpha_new = source_alpha[classclassify[3]]

### read in BH mass table
BHmass_table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS BH mass.csv')
BHmass_name = BHmass_table['Name'].values
BHmass = BHmass_table['BH_mass'].values

### find overlap source name
overlap_index = []
for i in range(0, len(BHmass_name)):
    index = np.where(source_name == BHmass_name[i] )[0]
    overlap_index.append(index[0])

### remove overlap sources 
source_name_new = list(source_name_new)
for i in range(0, len(overlap_index)):
    source_name_new.remove(source_name[overlap_index][i])
    
    

In [13]:
for zz in range(0, 1):  # len(source_name_new) = 821
    df = ClearData()
    
    name = source_name_new[zz]
    if z_new[zz] == 0:
        z = 0.01
    else:
        z = z_new[zz]
    #print('z = ', z, ', name: ', name)
    
    path2 = f'/Users/87steven/Downloads/FSRQ SED fit test/SED model fit plotting parameters/'+ name +' SED model fit/'
    if not os.path.isdir(path2):
        os.mkdir( path2 )
        
    #######################################################################################################
    ### read data
    #######################################################################################################
    # My source
    source = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/source individual flux 2023_2_15/' + name + '_flux.csv') 

    freq = source['freq'].values
    flux = source['flux'].values
    flux_err = source['flux_err'].values
    
    flux = flux*(1+z)**2   # K-correction

    ### set flux error which is nan to 0.01
    fluxerr_nan = np.where( np.isnan(flux_err ))[0]
    flux_err[fluxerr_nan] = 0.01
    ### find flux index which is not nan
    flux_NOTnan = np.where( (~np.isnan(flux)) & (flux > 0) )[0]
    ### save new data into array
    freq = freq[flux_NOTnan]
    flux = flux[flux_NOTnan]
    flux_err = flux_err[flux_NOTnan]

    flux_err_correct = flux_err/0.434*flux
    #######################################################################################################
    ### systematic errors
    #######################################################################################################
    # array of systematic errors, will just be summed in quadrature to the statistical error
    ### define energy ranges
    nu_vhe = 2.42E25 # [Hz]
    nu_he = 2.42E22 # [Hz]
    nu_x_ray_max = 4.25E19 # [Hz]
    nu_x_ray_min = 7.25E16 # [Hz]
    vhe_gamma = freq >= nu_vhe
    he_gamma = (freq >= nu_he) * (freq  < nu_vhe)
    x_ray = (freq  >= nu_x_ray_min) * (freq  < nu_x_ray_max)
    uv_to_radio = freq  < nu_x_ray_min

    ### declare systematics
    y_err_syst = np.zeros(len(freq ))
    y_err_syst[vhe_gamma] = 0.30
    y_err_syst[he_gamma] = 0.10
    y_err_syst[x_ray] = 0.10
    y_err_syst[uv_to_radio] = 0.05
    y_err_syst = flux * y_err_syst

    # define the data1D object containing it
    sed = data.Data1D("sed", freq, flux, staterror = flux_err_correct, syserror = y_err_syst)  # , syserror = y_err_syst

    #######################################################################################################
    # test parameters
    #######################################################################################################
    B_val = np.linspace(0.01, 1, 10)
    delta_D_val = np.array([1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    k_e_val = 10**np.linspace(-5, -9, num = 5) 
    t_var_val = np.array([1, 5, 10, 15, 20, 24])
    
    aa = 0
        
    for i in range(0, 10): # 0-10
        ### declare the model
        agnpy_ec = AgnpyEC()
        
        #######################################################################################################
        ### global parameters of the blob and the DT
        #######################################################################################################
        d_L = Distance(z = z).to("cm")                  # luminosity distance [cm] (calculate

        # disk
        L_disk = 6.7e45 * u.Unit("erg s-1")           # disk luminosity   (see 
        M_BH = 4.67E8 * M_sun                         # black hole mass   (?
        eta = 1/12                                    # accretion efficiency  (?
        m_dot = (L_disk / (eta * c ** 2)).to("g s-1") # accretion rate, c is speed of light (calculate
        R_g = ((G * M_BH) / c ** 2).to("cm")          # Schwarzschild radius [cm] (calculate
        R_in = 6 * R_g                                # disk inner radius [cm] (calculate
        R_out = 10000 * R_g                           # disk outer radius [cm] (calculate

        # Dust Torus (DT)
        xi_dt = 0.6                                   # fraction of disk luminosity reprocessed by the DT 
        T_dt = 1.0e3                                    # Temperature of the dust torus [K] 
        R_dt = 1.6E19                                # radius of the torus [cm] 
        
        if np.isnan(alpha_new[zz]):
            p1 = 2
            p2 = 2
        elif 2*source_alpha[zz]+1 > 5:
            p1 = 5
            p2 = 5
        else:
            p1 = 2*source_alpha[zz]+1
            p2 = 2*source_alpha[zz]+1
        
        ### free parameters
        log10_B = np.log10(B_val[i])                 # magnetic field strength (free
        #######################################################################################################
        ### find frequency of maximum flux between 1.0E12 ~ 1.0E20
        #######################################################################################################
        index = np.where((freq > 1.0E12) & (freq <= 1.0E20))[0]

        max_flux_index = np.where(flux == max(flux[index]))[0]
        min_freq = freq[0]
        max_freq = freq[max_flux_index][0]

        gamma_b_min = np.sqrt(min_freq/(4.2E6*B_val[i]))
        gamma_b_val = np.sqrt(max_freq/(4.2E6*B_val[i]))

        log10_gamma_b = np.log10(gamma_b_val)         # lorentz break factor (calculate
        log10_gamma_min = np.log10(gamma_b_min)       # minimum lorentz factor (calculate
        log10_gamma_max = np.log10(gamma_b_val*100)   # maximum lorentz factor (calculate
        
        for m in range(0, 5):  # 0-5
                log10_k_e = np.log10(k_e_val[m])         #  (-7.9 = np.log10(1.26E-8))

                for j in range(0, 6): # 0-24
                    t_var = 60*60*t_var_val[j]                   # variability timescale [sec] (86400)

                    for k in range(0, 11): 
                        df_2 = SED_Data()

                        delta_D = delta_D_val[k]                          # doppler factor (free parameter)
                        
                        ### blob
                        Gamma = delta_D_val[k]                             # Lorentz factor 
                        Beta = np.sqrt(1 - 1 / np.power(Gamma, 2))    # jet relativistic speed (calculate
                        mu_s = (1 - 1 / (Gamma * delta_D)) / Beta     # viewing angle (calculate
                        r = c*t_var*delta_D/(1+z)*100                 # size of emission region [cm] (calculate
                        r = r.value
                        if r < 1.0E16:
                            r = 1.0E16
                        
                        ### save plotting parameters
                        df_2 = SED_Data()

                        print('source name =' , name, ', # of run = ', aa)
                        print("i = ", i, ", m = ", m, ", j = ", j, ", k = ", k)
                        
                        # instance of the model wrapping angpy functionalities
                        # - AGN parameters
                        # -- distances
                        agnpy_ec.z = z                                # redshift 
                        agnpy_ec.z.freeze()
                        agnpy_ec.d_L = d_L.cgs.value                  # luminosity distance [cm]
                        agnpy_ec.d_L.freeze()
                        # -- SS disk
                        agnpy_ec.log10_L_disk = np.log10(L_disk.to_value("erg s-1"))  # disk luminosity
                        agnpy_ec.log10_L_disk.freeze()
                        agnpy_ec.log10_M_BH = np.log10(M_BH.to_value("g")) # black hole mass
                        agnpy_ec.log10_M_BH.freeze()
                        agnpy_ec.m_dot = m_dot.to_value("g s-1")      # accretion rate
                        agnpy_ec.m_dot.freeze()
                        agnpy_ec.R_in = R_in.to_value("cm")           # [cm]
                        agnpy_ec.R_in.freeze()
                        agnpy_ec.R_out = R_out.to_value("cm")         # [cm]
                        agnpy_ec.R_out.freeze()
                        # -- Dust Torus
                        agnpy_ec.xi_dt = xi_dt                        # fraction of disk luminosity reprocessed by the DT
                        agnpy_ec.xi_dt.freeze()
                        agnpy_ec.T_dt = T_dt                          # [K]
                        agnpy_ec.T_dt.freeze()
                        agnpy_ec.R_dt = R_dt                          # [cm]
                        agnpy_ec.R_dt.freeze() 
                        # - blob parameters
                        agnpy_ec.delta_D = delta_D                    # doppler factor δ
                        agnpy_ec.delta_D.freeze()
                        agnpy_ec.log10_B = log10_B                    # magnetic field sterength [G]
                        agnpy_ec.mu_s = mu_s                          # viewing angle
                        agnpy_ec.mu_s.freeze()
                        agnpy_ec.t_var = t_var                        # varaibility timescale of emission region [sec]
                        agnpy_ec.t_var.freeze()
                        agnpy_ec.log10_r = np.log10(r)                # size of emission region [cm]
                        agnpy_ec.log10_r.freeze()
                        # - EED
                        agnpy_ec.log10_k_e = log10_k_e                # opacity [cm^-3]
                        agnpy_ec.p1 = p1                              # electron distribution index
                        agnpy_ec.p2 = p2                              # electron distribution index
                        agnpy_ec.log10_gamma_b = log10_gamma_b        # lorentz break factor
                        agnpy_ec.log10_gamma_min = log10_gamma_min    # minimum lorentz factor
                        agnpy_ec.log10_gamma_min.freeze()
                        agnpy_ec.log10_gamma_max = log10_gamma_max     # minimum lorentz factor
                        agnpy_ec.log10_gamma_max.freeze()
                    
                        #######################################################################################################
                        ### fit using the Levenberg-Marquardt optimiser
                        #######################################################################################################
                        fitter = Fit(sed, agnpy_ec, stat = Chi2(), method = LevMar())

                        # Set minimum and maximum frequency to proced model fit
                        min_x = 1.0E7
                        max_x = 1.0E30
                        sed.notice(min_x, max_x)

                        #######################################################################################################
                        ### perform model fit
                        #######################################################################################################
                        result = fitter.fit()

                        print("Model fit succesful??", result.succeeded)
                        
                        #######################################################################################################
                        ### define the emission region and the thermal emitters
                        #######################################################################################################
                        k_e = 10**agnpy_ec.log10_k_e.val * u.Unit("cm-3")
                        p1 = agnpy_ec.p1.val
                        p2 = agnpy_ec.p2.val
                        gamma_b = 10**agnpy_ec.log10_gamma_b.val
                        gamma_min = 10**agnpy_ec.log10_gamma_min.val
                        gamma_max = 10**agnpy_ec.log10_gamma_max.val
                        B = 10**agnpy_ec.log10_B.val * u.G
                        r = 10**agnpy_ec.log10_r.val * u.cm
                        delta_D = agnpy_ec.delta_D.val
                        R_b = c.value * agnpy_ec.t_var.val * agnpy_ec.delta_D.val / (1 + agnpy_ec.z.val)*100

                        ### blob definition
                        parameters = {
                            "p1": p1,
                            "p2": p2,
                            "gamma_b": gamma_b,
                            "gamma_min": gamma_min,
                            "gamma_max": gamma_max,
                        }

                        spectrum_dict = {"type": "BrokenPowerLaw", "parameters": parameters}
                        blob = Blob(
                            R_b*u.cm,
                            z,
                            delta_D,
                            Gamma,
                            B,
                            k_e,
                            spectrum_dict,
                            spectrum_norm_type = "differential",
                            gamma_size = 500, 
                        )

                        #######################################################################################################
                        ### Disk and DT definition
                        #######################################################################################################
                        L_disk = 10 ** agnpy_ec.log10_L_disk.val * u.Unit("erg s-1")
                        M_BH = 10 ** agnpy_ec.log10_M_BH.val * u.Unit("g")
                        m_dot = agnpy_ec.m_dot.val * u.Unit("g s-1")
                        eta = (L_disk/(m_dot*c**2)).to_value("")
                        R_in = agnpy_ec.R_in.val * u.cm
                        R_out = agnpy_ec.R_out.val * u.cm
                        disk = SSDisk(M_BH, L_disk, eta, R_in, R_out)
                        dt = RingDustTorus(L_disk, xi_dt, T_dt * u.K, R_dt = R_dt * u.cm)

                        #######################################################################################################
                        ### define plotting parameters
                        ######################################################################################################
                        synch = Synchrotron(blob, ssa = True)
                        ssc = SynchrotronSelfCompton(blob, synch)
                        ec_dt = ExternalCompton(blob, dt, r)
                        # SEDs
                        nu = np.logspace(7, 30, 200) * u.Hz
                        synch_sed = synch.sed_flux(nu)
                        ssc_sed = ssc.sed_flux(nu)
                        ec_dt_sed = ec_dt.sed_flux(nu)
                        disk_bb_sed = disk.sed_flux(nu, z)
                        dt_bb_sed = dt.sed_flux(nu, z)
                        total_sed = synch_sed + ssc_sed + ec_dt_sed + disk_bb_sed + dt_bb_sed
                        
                        data_x = sed.x
                        data_y = sed.y
                        y_err_lin = sed.get_error()

                        fit_x = np.logspace(np.log10(min_x), np.log10(max_x), 200)
                        fit_y = agnpy_ec(fit_x).to_value()

                        #######################################################################################################
                        ### save model fit parameters
                        #######################################################################################################
                        #for n in range(0, len(k_e_array)):
                        df['i'].append(i) 
                        df['m'].append(m)
                        df['j'].append(j)
                        df['k'].append(k) 
                        df['p1'].append(p1)
                        df['p2'].append(p2)
                        df['k_e'].append(k_e)
                        df['gamma_b'].append(gamma_b) 
                        df['gamma_min'].append(gamma_min)
                        df['gamma_max'].append(gamma_max) 
                        df['B'].append(B)
                        df['delta_D'].append(delta_D)
                        df['R_b'].append(R_b) 
                        df['chi_2'].append(float(result.rstat))
                        df['V_b'].append(blob.V_b)
                        df['t_var'].append(blob.t_var.to_value("s")) 
                        df['d_L'].append(blob.d_L)
                        df['Gamma'].append(blob.Gamma) 
                        df['Beta'].append(blob.Beta)
                        df['xi'].append(blob.xi)
                        df['theta_s'].append(blob.theta_s) 
                        df['xi_dt'].append(xi_dt)   # fraction of the disk radiation reprocessed by the torus
                        df['T_dt'].append(T_dt)   # temperature of the dust torus
                        df['R_dt'].append(R_dt)   # radius of the torus
                        
                        CSVfile = f'/Users/87steven/Downloads/FSRQ SED fit test/SED model fit parameters/' + name + '_SED_model fit parameters.csv'                    
                        dff = pd.DataFrame(df)     
                        dff.to_csv(CSVfile, index = False) 

                        ### save figure parameters
                        for ii in range(0, len(fit_x)):
                            df_2['fit_x'].append(fit_x[ii])
                            df_2['fit_y'].append(fit_y[ii])
                            df_2['syn_y'].append(synch_sed.value[ii])
                            df_2['ssc_y'].append(ssc_sed.value[ii])
                            df_2['ec_dt_y'].append(ec_dt_sed.value[ii])
                            df_2['disk_bb_y'].append(disk_bb_sed.value[ii])
                            df_2['dt_bb_y'].append(dt_bb_sed.value[ii])

                        CSVfile = f'/Users/87steven/Downloads/FSRQ SED fit test/SED model fit plotting parameters/'+ name +' SED model fit/' + name + '_' + str(aa) +'.csv'                       
                        dff = pd.DataFrame(df_2)     
                        dff.to_csv(CSVfile, index = False)
                        
                        aa = aa+1
                        
                        print('========== File saved successfully ==========')


source name = J1930-6056 , # of run =  0
i =  0 , m =  0 , j =  0 , k =  0
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  1
i =  0 , m =  0 , j =  0 , k =  1
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  2
i =  0 , m =  0 , j =  0 , k =  2
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  3
i =  0 , m =  0 , j =  0 , k =  3
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  4
i =  0 , m =  0 , j =  0 , k =  4
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  5
i =  0 , m =  0 , j =  1 , k =  0
Model fit succesful?? True
========== File saved successfully ==========
source name = J1930-6056 , # of run =  6
i =  0 , m =  0 , j =  1 , k =  1
Model fit succesful?? True
==========


KeyboardInterrupt

